In [2]:
!pip install pymilvus sentence_transformers

  Using cached grpcio-1.67.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.9 kB)
  Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.3 kB)
  Using cached milvus_lite-2.4.12-py3-none-macosx_11_0_arm64.whl.metadata (10.0 kB)
Using cached grpcio-1.67.1-cp311-cp311-macosx_10_9_universal2.whl (11.0 MB)
Using cached milvus_lite-2.4.12-py3-none-macosx_11_0_arm64.whl (17.4 MB)
Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl (51 kB)
  Attempting uninstall: grpcio0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
    Found existing installation: grpcio 1.71.0━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
    Uninstalling grpcio-1.71.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
      Successfully uninstalled grpcio-1.71.0━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pymilvus]3/4 [pymilvus]


In [1]:
from sentence_transformers import SentenceTransformer
import torch

# 디바이스 설정: MPS 사용
device = "mps" if torch.backends.mps.is_available() else "cpu"
# device = "cpu"
# 모델 로드 및 디바이스 이동
model = SentenceTransformer("BAAI/bge-m3")
model.to(device)

/Users/ranifarm/.conda/envs/adaptive-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
from pymilvus import MilvusClient, DataType

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"